In [2]:
import numpy as np

# Statistical procedures

A substantial proportion of real world applications in computational modelling require statistical procedures. `numpy` provides a wide variety of efficient statistical functions for you to employ on an array.  This section will explore the (simple and) commonly used functions that you can embed into your models and use for practical statistical analysis.  At the end of the section I'll tell a cautionary tale about blind use of `numpy` statistical procedures versus using a well design algorithm.

<div class="alert alert-block alert-info"><b>Tip:</b> We will explore statistical programming for health data science in a lot more detail in Part 2 using `pandas` and other important libraries.  It is well worth learning `numpy` capabilities, however, as converting from a `np.ndarray` to a `pandas.DataFrame` during a computational procedure can be expensive. </div>



## Simple data analysis example.

### ED attendance data

We will first use data held in the `minor_illness_ed_attends.csv`.  This is a synthetic time series dataset reporting the number of patients registered at GP surgery who attend ED each week.  The data are standardised to 10k of registered patients.

#### Loading the dataset

Let's first open the data and then construct some summary statistics

In [272]:
file_name = 'data/minor_illness_ed_attends.csv'
ed_data = np.loadtxt(file_name, skiprows=1, delimiter=',')
print(ed_data.shape)

(74,)


Here's a peak the first 5 elements in `ed_data`.

In [273]:
ed_data[:5]

array([2.11927795, 3.49057545, 3.98922908, 2.36860477, 3.24124863])

#### Calculate summary statistics

`numpy` makes it easy to calculate means, stdev and other summary statistics of an `ndarray`.  This typically have intuitive names which you can often guess. For example:

In [274]:
print(ed_data.mean())
print(ed_data.std())

2.919482262743243
0.7060975938853894


But it is always worth reading the docs.  For example to calculate an unbiased estimate of the sample standard deviation of a data set of length $n$ we should divide by $n - 1$.  In `numpy` we do this using the `ddof` (degree's of freedom) parameter of `std`.

Here we will create a class to act as a convienient container for a dataset.  For convenience, we will override the `__str__` method so that we can easily print a summary of the dataset to the screen when calling `print`.

The class will make use of the following `numpy` statistical procedures:

* `min()` and `max()` to return the minimum and maximum value in array respectively
* `np.percentile()` to calculate a percentile - here the median and upper / lower quartiles. 
* `np.histogram()` that bins data points and reports the frequencies.

In [315]:
class AttendanceSummary:
    '''
    Simple container class Hold mean, stdev and 5/95 percentiles of ed data
    '''
    def __init__(self, data, name=None, decimal_places=2):
        """
        Contructor method.
        
        Parameters:
        ----------
        data: numpy.ndarray 
            Vector containing data to analyse.
        """
        self.name = name
        self.dp = decimal_places
        self.n = len(data)
        self.mean = data.mean()
        self.std = data.std(ddof=1)
        self.min_attends = data.min()
        self.max_attends = data.max() 
        
        # percentiles: note that this is a np. call
        self.lower_quartile = np.percentile(data, 25)
        self.median = np.percentile(data, 50)
        self.upper_quartile = np.percentile(data, 75)
        self.iqr = self.upper_quartile - self.lower_quartile
        
        # frequency histogram (automatically binned)
        self.freq, self.bins = np.histogram(data, density=False)
        
    def __repr__(self):
        to_print = f'\nDataset: {self.name}' \
             + f'\nMean:\t{self.mean:.{self.dp}f}' \
             + f'\nStdev:\t{self.std:.{self.dp}f}' \
             + f'\nMin:\t{self.min_attends:.{self.dp}f}' \
             + f'\nMax:\t{self.max_attends:.{self.dp}f}' \
             + f'\nMedian:\t{self.median:.{self.dp}f}' \
             + f'\nIQR:\t{self.iqr:.{self.dp}f}' \
             + f'\nn:\t{self.n}'
                    
        return to_print
    
    def frequency_histogram(self):
        print('x\tf(x)')
        for f, b in zip(self.freq, self.bins):
            print(f'{b:.{self.dp}f}\t{f}')

In [316]:
x = AttendanceSummary(ed_data, name="ED")
x


Dataset: ED
Mean:	2.92
Stdev:	0.71
Min:	1.62
Max:	5.11
Median:	2.87
IQR:	1.09
n:	74

In [317]:
x.frequency_histogram()

x	f(x)
1.62	5
1.97	9
2.32	14
2.67	16
3.02	11
3.37	6
3.71	10
4.06	2
4.41	0
4.76	1


Its not necessary to always use a class as I here, but it is useful if for example you want to compare samples or in the case of a time series period.

In our ED data let's assume an intervention was put in place in week 10: GP surgeries were opened to patients over weekends. Let's summarise the results using descriptive statistics.

In [318]:
week = 10  # the week the intervention begins
before = AttendanceSummary(ed_data[:week], 'before')
after = AttendanceSummary(ed_data[week:], 'after')
print(before)
print(after)


Dataset: before
Mean:	3.12
Stdev:	0.59
Min:	2.12
Max:	3.99
Median:	3.18
IQR:	0.81
n:	10

Dataset: after
Mean:	2.89
Stdev:	0.73
Min:	1.62
Max:	5.11
Median:	2.87
IQR:	0.90
n:	64


## Statistical procedures and n-dimensional arrays

Generating statistics for matricies and n-dimensional arrays works in a similar manner to vectors.  To do this you need to specify an **axis** in the call to the statistical function e.g. `mean()`

### Example: mean of matrix columns and rows

Given `matrix` calculate the mean of the columns and rows.

In [343]:
matrix = np.array([[7.7, 4.3, 8.5],
                   [6.9, 0.9, 9.7],
                   [7.6, 7.8, 1.2]])

In [335]:
matrix[:,0].mean()

7.400000000000001

In [336]:
matrix[:,1].mean()

4.333333333333333

In [337]:
matrix[:,2].mean()

6.466666666666666

In [342]:
matrix.mean(axis=0).round(1)

array([7.4, 4.3, 6.5])

## A note of caution: working with running statistics

In many computational modelling procedures you will need an estimate of statistics as the code executes. For example, you may need to track a mean or a standard deviation of a performance measure in a multi-stage algorithm or as a simulation model of a healthcare system executes. 

As we have seen `numpy` provides highly efficient functions for calculating a mean or standard deviation based on data held in an array.  I'm always tempted to make use of these built in procedures. They are indeed fast and incredibly easy to use.  The downside is that you waste computation via repeated iteration over an array.  The other option, that requires more careful thought (due to floating point issues), is a running estimate of your statistics.  In general, I've implemented such procedures  in standard python.  Let's look at an example where we compare recalculation using a `numpy` function with a running (sometimes called an 'online') calculation of the mean and standard deviation in standard python.

We will first refactor `AttendanceSummary` to an `OnlineSummary` class to include an `update()` function.  It will accept a `np.ndarray` that recalculates the **sample** mean and standard deviation using a `numpy` on the full data set.  The function `test_complete_recalculation` iteratively calls `update` using more data each time.  For simplicities sake we will reuse the data contained within `ed_data`.

In [257]:
class OnlineSummary:
    
    def __init__(self, data=None, decimal_places=2):
        """
        Track online statistics of mean and standard deviation.

        Params:
        -------
        data: np.ndarray, optional (default = None) 
            Contains an initial data sample.
            
        decimal_places: int, optional (default=2)
            Summary decimal places.
        """
        if isinstance(data, np.ndarray):
            self.n = len(data)
            self.mean = data.mean()
            self.std = data.std(ddof=1)
        else:
            self.n = 0
            self.mean = None
            self.std = None
            
        self.dp = decimal_places
        
    def update(self, data):
        '''
        Update the mean and standard deviation using complete recalculation.
        
        Params:
        ------
        data: np.ndarray
            Vector of data
        '''
        self.n = len(data)
        
        # update the mean and std. Easy!
        self.mean = data.mean()
        self.std = data.std(ddof=1)
        
    
    def __str__(self):
        to_print = f'Mean:\t{self.mean:.{self.dp}f}' \
             + f'\nStdev:\t{self.std:.{self.dp}f}' \
             + f'\nn:\t{self.n}' \
        
        return to_print

In [258]:
def test_complete_recalculation(data, start=1):
    summary = OnlineSummary(data[:start])

    for i in range(start, len(data)+1):
        summary.update(data[:i])
    return summary

In [259]:
summary = test_complete_recalculation(ed_data)
print(summary)

Mean:	2.92
Stdev:	0.71
n:	74


In [260]:
%timeit summary = test_complete_recalculation(ed_data)

1.59 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


You should find the `numpy` implementation fairly efficient clocking in at around 1.5ms on average. But can we do better in standard python by computing an online mean and standard deviation?

To do this we will use **Welford's algorithm** for computing a running sample mean and standard deviation.  This is a robust, accuate and old(ish) approach (1960s) that I first read about in Donald Knuth's *art of computer programming vol 2.* (just to be clear I learnt how to do this in 2008 not 1960!).  To implement it we need to refactor `update`.  Note that we will need a fair bit more code than our simple `numpy` solution.

The algorithm is given in a recursive format.  For our purposes here, you can just think of that as tracking the mean and standard deviation as attributes of a class that we iteratively update with a new $x$. 

The first thing you need to do is handle the first observation encountered. 

$$M_1 = x_1$$
$$S_1 = 0$$

Then on each subsequent call you update $M$ and $S$ making use of the previous values.  Note that $M$ has a relatively simple interpretation: its the sample mean. However, $S$ is not the standard deviation.  Its actually the sum of squares of differences from the current mean.  We will look at how to update that first and then I'll show you the equation for converting to the standard deviation.

$$M_n = M_{n-1} + \dfrac{x_n - M_{n-1}}{n}$$


$$S_n = S_{n-1} + \left[(x_n - M_{n-1}) \times (x_n - M_n)\right]$$

If the equations are confusing you can think of $M_n$ as the `updated_mean` and $M_{n-1}$ as the `previous_mean`.

Once the update is complete it is then relatively trivial to calculate the standard deviation $\sigma_n$.  Note that we don't necessarily need to track the standard deviation just $S_n$.  We can inexpensively calculate $\sigma_n$ when it is needed.  

$$\sigma_n = \sqrt{\dfrac{S_n}{n-1}}$$

The code listing below modifies `OnlineSummary` to make use of Welford's algorithm. Note that `std` is now a property that calculates the standard deviation on the fly using $S_n$

In [261]:
class OnlineSummary:
    
    def __init__(self, data=None, decimal_places=2):
        """
        Returns mean, stdev and 5/95 percentiles of ed data

        Params:
        -------
        data: np.ndarray, optional (default = None) 
            Contains an initial data sample.
            
        decimal_places: int, optional (default=2)
            Summary decimal places.
        """
        
        self.n = 0
        self.mean = None
        self._sq = None
        
        if isinstance(data, np.ndarray):
            for x in data:
                self.update(x)
            
        self.dp = decimal_places
    
    @property
    def variance(self):
        return self._sq / (self.n - 1)
    
    @property
    def std(self):
        return np.sqrt(self.variance)
    
    def update(self, x):
        '''
        Running update of mean and variance implemented using Welford's 
        algorithm (1962).
        
        See Knuth. D `The Art of Computer Programming` Vol 2. 2nd ed. Page 216.
        
        Params:
        ------
        x: float
            A new observation
        '''
        self.n += 1
        
        # we need to do more work ourselves for online stats!
        
        # init values
        if self.n == 1:
            self.mean = x
            self._sq = 0
        else:
            # compute the updated mean
            updated_mean = self.mean + ((x - self.mean) / self.n)
        
            # update the sum of squares 
            self._sq += (x - self.mean) * (x - updated_mean)
            
            # update the tracked mean
            self.mean = updated_mean
    
    def __str__(self):
        to_print = f'Mean:\t{self.mean:.{self.dp}f}' \
             + f'\nStdev:\t{self.std:.{self.dp}f}' \
             + f'\nn:\t{self.n}' \
        
        return to_print

In [264]:
def test_online_calculation(data, start=1):
    summary = OnlineSummary()

    for observation in data:
        summary.update(observation)
    return summary

In [265]:
summary = test_online_calculation(ed_data)
print(summary)

Mean:	2.92
Stdev:	0.71
n:	74


In [266]:
%timeit summary = test_online_calculation(ed_data)

44.8 µs ± 597 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Crickey nothing beats a good algorithm! You should find that you are now working in microseconds (µs) as opposed to milliseconds. 1µs = 1000ms. On my machine the `test_online_calculation` executes in ~45 µs on average while `test_complete_recalculation` takes ~1500 µs. So we are finding a speed up of ~97%. That gap will continue to grow as the number of samples $n$ increases.  The result is explained because our second implementation has a constant time for execution (and constant number of computational steps) while the time complexity of the `numpy` call depends on the size of the array. That's a lesson well worth remembering when developing code for scientific applications requiring performant code.